## Postulates Golomb
The postulates Golomb are neccesary but not sufficient conditions for pseudorandom sequences to appear random.

Its operation is simple:

Give a sequence $S=\{S^0,S^1,...,S^n$ of period **n**, the postulates they pose are:

1. In the cycle $S^n$ of **S**, the difference of 1s and 0s that there is at most has to be 1.
2. In the cycle $S^n$, the various runs are of length $\frac{1}{2^n}$ exactly, i.e. half runs of length 1, quarter runs of length 2, eighth runs of length 3, ...
3. We cyclically shift the sequence $S^n$ by one bit and calculate the Hamming distance between the new sequence and the original, if the distance is the same as the previous one, the process is repeated, in otherwise it ends and it is verified that the last value obtained is 0. If it is 0, it fulfills the third postulate, otherwise it does not. 

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Apr  6 18:14:10 2022

@author: Ruben Girela Castellón
"""
import numpy as np

In [2]:
#función que determina si una secuencia de bits cumple los postulados de Golomb
def postGolomb(bits):
    
    #1º comprobamos que la cantidad de 1s y 0s difieren como mucho en 1
    #para ello contamos el numero de 0s y 1s
    ceros = bits.count('0')
    unos = bits.count('1')
    
    #compruebo la diferencia absoluta entre 0s y 1s sean 0 o 1 como mucho.
    if(np.abs(ceros-unos) <= 1):
        
        '''
        iniciamos la racha anterior y la siguiente como el maximo entre los 
        contadores de 0s y 1s y rachas_next como la mitad de la anterior.
        '''
        rachas_last = max(ceros, unos)
        rachas_next = rachas_last//2
        
        #contador en el que el numero de n-rachas sea 1
        contador_unos = 0
        
        #iniciamos el numero de rachas a 0
        rachas = 0     
        
        #comprobamos si el primer bit y el ultimo no son distintos
        while(bits[0] == bits[-1]):
            #si son iguales rotamos hasta que el primero sea distinto al ultimo
            bits = bits[-1]+bits[:-1]
        

        '''
        2º comprobamos de las diversas rachas son de longitud 1/2^n, siendo n el 
        numero de rachas, mientras el contador de n-rachas siguiente sea >= que 
        la mitad entera del anterior o el contador de unos es > 0 y < 3 y el 
        contador de n-rachas siguientes no sea 0.
        '''
        while(
                (rachas_next == rachas_last//2 or 
                 (contador_unos < 3 and contador_unos > 0)
                ) and rachas_next > 0):
            
            #si el numero de n-rachas es 1 incrementamos el contador de unos
            if(rachas_next == 1): contador_unos +=1
            
            #incremento el numero de rachas
            rachas += 1
            #contador de rachas que será modificado
            rachas_m = rachas
            
            #si el numero de rachas es 2 o más
            if(rachas > 1):
                #actualizo rachas_last
                rachas_last = rachas_next
                
            #reseteamos rachas_next
            rachas_next = 0
                
            #inicializamos el posicionador de bits
            i = 0
            
            #mientras no supere el rango de la secuencia de bits
            while(i<len(bits)):
                
                #obtengo el primer bit distinto con el anterior
                bit = bits[i]
                    
                #incrementamos i
                i += 1

                #decrementamos el contador de rachas
                rachas_m -= 1

                #si no supera el rango de bits
                if(i < len(bits)):

                    '''
                    si el numero de rachas es 2 o mas o el bit siguiente es 
                    igual al anterior.
                    '''
                    if(rachas_m>0 or bits[i] == bit):

                        #avanza hasta el siguiente bit distinto
                        while(i < len(bits) and bits[i]==bit):

                            #decrementando el contador de rachas
                            rachas_m -= 1

                            #e incrementando la posición
                            i += 1

                #si el numero de rachas es 0
                if(rachas_m == 0):
                    #incrementamos el contador de n-rachas
                    rachas_next += 1

                #reseteamos el contador de rachas
                rachas_m = rachas  
            
            #si la racha siguiente es impar y es mayor a 1, no cumple el 2º postulado
            if(rachas_next%2 != 0 and rachas_next > 1): return False
            
            #si el numero de n-rachas es 0 y el contador de unos no supera a 2
            if(rachas_next == 0 and contador_unos <= 2 and rachas > 1):
                
                '''
                Cuando se cumple el 2º postulado, comprobamos que se cumple el 
                3º postulado, que consta de desplazar un bit de forma ciclica y 
                calcular la distancia de Hamming de esa nueva secuencia con la 
                original, se compara la distancia Hamming anterior con la siguiente
                , si son iguales se repite el proceso, hasta que sean distintos, 
                si es 0 cumple el 3º postulado, en caso contrario no.
                '''
                #para ello desplazamos un bit a la derecha.
                #Ejemplo: 1001 --> 0011
                #convertimos la secuencia en un array de enteros
                b1 = np.array(list(map(int,bits)))
                #desplazo la secuencia de derecha a izquierda
                b2 = b1[1:]
                b2 = np.append(b2,b1[0])
                
                #calculo la primera diferencia
                after_diff = np.sum(np.abs(b1-b2))
                #se pone el mismo valor que el anterior para que entre en el bucle
                diff = after_diff
                
                '''
                mientras sea la diferencia igual a la anterior itera hasta que 
                sea distinto:
                    - 0 (ha dado la vuelta a toda la secuencia)
                    - x (no cumple el 3º postulado)
                '''
                while(after_diff == diff):
                    
                    #desplazo la secuencia de derecha a izquierda
                    b2 = np.append(b2,b2[0])
                    b2 = b2[1:]
                    
                    #calculo la distancia hamming
                    diff = np.sum(np.abs(b1-b2))
                    
                #si la diferencia es 0 es que cumple los postulados de Golomb
                if(diff == 0): return True
                #en caso contrario no
                return False
    
    #en caso contrario no
    return False

Some examples:

In [3]:
bits = ['000111101011001','100011', '1000001', '00101001110110', '0001101',
        '00011110101100100011110101100']

for i in bits:
    pg = postGolomb(i)
    if(pg):
        print(f'({i}) cumple con los postulados de Golomb')
    else:
        print(f'({i}) no cumple con los postulados de Golomb')

(000111101011001) cumple con los postulados de Golomb
(100011) no cumple con los postulados de Golomb
(1000001) no cumple con los postulados de Golomb
(00101001110110) no cumple con los postulados de Golomb
(0001101) cumple con los postulados de Golomb
(00011110101100100011110101100) no cumple con los postulados de Golomb


## Linear Feedback Shift Register (LSFR)
**LSFR** is a shift register, in which the input is a bit that comes from applying linear transform function to a previous state.

The initial value is called seed, and the way of operating is deterministic, on the other hand, the secuence of generated values are determined by current state and previous state.

### How does it work?
The function reveives the coefficients, seed and the length of the sequence to generate **L**.

Then we get the last **k bits** of the seed, where **k** is the length of the coefficients.

Example: 

If the coefficients are $c = x^{10} + x^9 + x + 1$ and the length of the coefficients is $k = 10$, then we will take the last 10 bits of the seed, since the coefficients in bits are **1100000001**.

Next, we multiply the last **k bits** of the binary seed by the binary coefficients and add all the bits to said result, obtaining a new bit that is added to the end of the seed sequence.

Example:
$$seed = 1010; \thinspace \thinspace coefficients = X^4 + X + 1 = 1001;$$
$$new\_bit = \sum{1010 * 1001} = \sum{1000} = 1$$

This operation is repeated until the length of the output sequence is equal to **L**, returning a cyclic sequence.

In [4]:
def LFSR(coeficientes, semilla, L):
    
    #secuencia final a devolver, que contendra semilla + restoBitsGenerados
    secuencia_f = semilla
    
    #contador de bits nuevos
    contador = 0
    
    #contador total que tiene actualmente la secuencia de salida
    contadorTotal = len(secuencia_f)
        
    #entra en un bucle infinito
    while(contadorTotal < L):
        
        '''
        genero el resto de bits con los coeficientes y los ultimos x valores
        de la semilla de longitud igual a la de los coeficientes (k).
        Ejemplo:
        # c -->  1001
        # s --> 01101 --> 1*1 + 0*1 + 0*0 + 1*1 = 0
        '''
        #cojo los ultimos k bits de la semilla.
        ultimos = np.array(list(map(int,secuencia_f[-len(coeficientes):])))
        
        '''
        Obtengo el nuevo bit a traves del producto escalar 
        sum(c*ultimos_k_de_s) en modulo 2
        '''
        bit = np.sum(ultimos * np.array(list(map(int,coeficientes))))%2
        
        #Añado el nuevo bit a la secuencia
        secuencia_f += str(bit)
        
        #incremento el contador de que se ha añadido un nuevo bit
        contador += 1
        
        #Cuando el contador coincide con la longitud de los coeficientes (L)
        if(contador == len(semilla)):
            #reseteo el contador
            contador = 0
            
            #incremento el contador en 4
            contadorTotal += 4
            
            '''
            y compruebo si ha llegado al final de la secuencia ciclica,
            para saber si ha llegado al final de la secuencia ciclica, 
            tiene que coincidir con el valor de los primeros L bits de la
            secuencia final con los ultimos L bits obtenidos de dicha secuencia.
            '''
            if(semilla == secuencia_f[-len(semilla):]):
                #si lo es devuelvo la secuencia ciclica
                return secuencia_f

Some examples:

Case of a reducible polynomial and therefore not primitive.

$$C = x^{4} + x^{3} + x^{2} + 1$$

$$L = 4; S = 7$$

In such a way that as can be seen in the result below, it generates a cyclic sequence, but not a complete one:

- 7, 4, 14, 9, 13, 3, 10

But, what if the seed is 1?

We obtain another different cyclic sequence, with other values different from the previous sequence.

- 1, 6, 2, 12, 5, 8, 11

Do all the values obtained from the 2 sequences, we are missing 0 and 15, in this case the values are repeated:

- 0, 0, ...
- 15, 15, ...

This tells us that it depends on the seed, we get one sequence or another. On the other hand, as the polynomial is reducible, it can be reduced to $C = (x+1)(x^3 + x + 1)$, indicating that it has 2 degrees, 1 and 3, with which we obtain cyclic sequences of length $2^1-1 = 1$ and $2^3-1=7$.


In [5]:
#Ejemplo polinomio reducible y por tanto no primitivo
coeficientes = '1110' #x^4 + x^3 + x^2 + 1
semilla = ['0111','0001', '1111', '0000']
L = [8*4, 8*4, 2*4, 2*4]

print('Ejemplo polinomio reducible y por tanto no primitivo:')
for i in np.arange(len(semilla)):
    secuencia = LFSR(coeficientes, semilla[i], L[i])

    print(f'Partiendo de los coeficientes {coeficientes}, semilla {semilla[i]}, y la longitud {L[i]}')
    print('Secuencia final: ',end='')
    for i in np.arange(len(secuencia), step=4):
        
        print(f'{secuencia[i-4:i]} ',end='')
    print(f'{secuencia[-4:]}')

Ejemplo polinomio reducible y por tanto no primitivo:
Partiendo de los coeficientes 1110, semilla 0111, y la longitud 32
Secuencia final:  0111 0100 1110 1001 1101 0011 1010 0111
Partiendo de los coeficientes 1110, semilla 0001, y la longitud 32
Secuencia final:  0001 0110 0010 1100 0101 1000 1011 0001
Partiendo de los coeficientes 1110, semilla 1111, y la longitud 8
Secuencia final:  1111 1111
Partiendo de los coeficientes 1110, semilla 0000, y la longitud 8
Secuencia final:  0000 0000


Case of a irreducible polynomial and not primitive

## Sources:
- https://es.wikipedia.org/wiki/Postulados_de_Golomb
- https://es.wikipedia.org/wiki/LFSR